In [1]:
include("src/lisa_store.jl")

using ..Store
using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using HDF5
using EasyConfig

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
h5open("lisa_arch.hdf5", "cw") do f
    # The file is now open, but it's empty
end

In [2]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/")

In [3]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row)
    col_uuid = string(uuid4())
    Store.ingest_csv_by_column(db, assign, col_uuid; limit=100, offset=10)
end

edge: Edge(0b90b1fee69c77ffa3efe57db7788112ef96dba6, 4afe2a681d14669b1148f259f7325220bab8ab2d, "has_column"; source="/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv", target="Severity of casualty", source_label="csv_file", target_label="csv_column")
edge: Edge(0b90b1fee69c77ffa3efe57db7788112ef96dba6, 48d3028983aa5e6b7c3bedefeda686acacfb1257, "has_column"; source="/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv", target="Casualty Class", source_label="csv_file", target_label="csv_column")
edge: Edge(0b90b1fee69c77ffa3efe57db7788112ef96dba6, 593c66c6d82d797a818caee475cd32abe120db76, "has_column"; source="/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv", target="Sex", source_label="csv_file", target_label="csv_column")
edge: Edge(0b90b1fee69c77ffa3efe57db7788112ef96dba6, 5b3f4a68cdaf988b8f353b916ff9cfc9c95466f3, "has_column"; source="/home/alexmy/JULIA/DEMO/sample/Isc_london-ksi-only-since2010.csv", target="Date", source_label="csv_file"

In [4]:
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

Removed Node: 0b90b1fee69c77ffa3efe57db7788112ef96dba6
Removed Node: 4afe2a681d14669b1148f259f7325220bab8ab2d
Removed Node: 48d3028983aa5e6b7c3bedefeda686acacfb1257
Removed Node: 593c66c6d82d797a818caee475cd32abe120db76
Removed Node: 5b3f4a68cdaf988b8f353b916ff9cfc9c95466f3
Removed Node: 8ee8a041ff42b775ac295f7e4c25860cbda7c00e
Removed Node: edad78a039f8c0975bfda5942d79e4aa35be1ccd
Removed Node: 3f9526f8d331b9519b8632a11b2d344ab7c647b6
Removed Node: 8e5610e809bad1efe796848a86ecb9e28e1709b9
Removed Node: 6be12bee4edf7c96016907e44bb520be80dc9232
Removed Node: 08d00fa7fb80c1af0f00777d3805b4f18f5a9281
Removed Node: 18d9d853910a574572553ab24e49045b9c18b5f2
Removed Node: 7699d7cece21e1d66b8deaed5cac93e650367ee6
Removed Node: 97ef61255a8f973052aa48a8db3d4ad57b357966
Removed Node: 83844dc0da738f19adbb5976e272c117fdc0f24c
Removed Node: 955d453e0d2df23ac7fb39749079d3c4cff7dad0
Removed Node: 350435b1d320f70f559162149ef1509f4485df47
Removed Node: abe1c6f48f6122e051e90390122b74ba87523bf4
Removed No

In [5]:
hdf = h5open("lisa_arch.hdf5", "r") 

🗂️ HDF5.File: (read-only) lisa_arch.hdf5

In [6]:
# Close the file
close(hdf)

In [7]:
h5open("lisa_arch.hdf5", "r") do h5f
    try
        read(h5f["/2da19026-e0ca-4729-86af-d183f5aec3ea/nodes/0b90b1fee69c77ffa3efe57db7788112ef96dba6"])
    catch e
        println(e)
    end
end

KeyError("/2da19026-e0ca-4729-86af-d183f5aec3ea/nodes/0b90b1fee69c77ffa3efe57db7788112ef96dba6")
